In [1]:
%run ../Python_files/util.py
%run ../Python_files/load_dicts.py

No dicts found; please check load_dicts...


In [2]:
tmc_capac_dict_AM['129-04138'] * 2.5, \
tmc_capac_dict_MD['129-04138'] * 4.75, \
tmc_capac_dict_PM['129-04138'] * 2.5, \
tmc_capac_dict_NT['129-04138'] * 7

(17500.0, 28500.0, 15000.0, 42000.0)

## raw data from CTPS capacity dataset

ID	LENGTH	DIR	ANODE	BNODE	CTPS_FUNCT	STREETNAME	ROUTENUMBE	SCEN_00_LA	SCEN_00_AB	SCEN_00_BA	SCEN_00_A1	SCEN_00_B1	SCEN_00_A2	SCEN_00_B2	SCEN_00_A3	SCEN_00_B3	AB_AMCAPAC	BA_AMCAPAC	AB_MDCAPAC	BA_MDCAPAC	AB_PMCAPAC	BA_PMCAPAC	AB_NTCAPAC	BA_NTCAPAC	ROADINVENT

56573	0.260468	1	73398	73394	1	INTERSTATE 93	93	2000	3.5	1	3	1	3	1	3	1	17500	5000	28500	9500	15000	5000	42000	14000	1132000

In [3]:
filtered_tmc_speed_dict = zload('../temp_files/tmc_speed_dict_for_anomaly_detection.pkz')

In [4]:
tmc = '129-04138'
month = 1
day = 9

for hour in range(24):
    for minute in range(60):
        key = str(tmc) + '_' + str(month) + '_' + str(day) + '_' + str(hour) + '_' + str(minute)
        
#         print(hour, minute, float(filtered_tmc_speed_dict[key].split('_')[0]))

In [5]:
tmc_ref_speed_dict[tmc]

73.0

In [6]:
from sympy import Symbol, nsolve
import sympy
x = Symbol('x')
nsolve(73*x - x**2 - 315, 0)

mpf('4.6056431323658762')

## Simulating a car accident
The road segment has three lanes, and an accident happened in one of the lanes, causing a sudden slow-down of the traffic. The instant flow would not change, while the capacity ($m$) would be reduced to be two thirds of the original value. Thus, using Greenshield's model, we have
$$4\left( {\frac{2}{3}m} \right)\left[ {\frac{{{v_2}}}{{{v_0}}} - {{\left( {\frac{{{v_2}}}{{{v_0}}}} \right)}^2}} \right] = 4m\left[ {\frac{{{v_1}}}{{{v_0}}} - {{\left( {\frac{{{v_1}}}{{{v_0}}}} \right)}^2}} \right], \quad (1)$$
where $v_1 = 70$, and $v_0 = 73$.

Solving (1), we obtain $v_2 = 4.6056431323658762$. Note that we only care about the road with heavy congestion in this case.

Assume that the accident happened during 17:15 - 17:45, Jan. 9, 2012.

In [7]:
tmc = '129-04138'
month = 1
day = 9

traffic_data_with_anomaly = {}

for hour in range(24):
    for minute in range(60):
        key = str(tmc) + '_' + str(month) + '_' + str(day) + '_' + str(hour) + '_' + str(minute)
        traffic_data_with_anomaly[key] = float(filtered_tmc_speed_dict[key].split('_')[0])
        if hour == 17 and minute > 15 and minute < 46:
            traffic_data_with_anomaly[key] = 4.6056431323658762
#         print(hour, minute, traffic_data_with_anomaly[key])

zdump(traffic_data_with_anomaly, '../temp_files/traffic_data_with_anomaly.pkz')

In [8]:
# extract reference traffic data, for the purpose of estimating PLs

tmc = '129-04138'
month = 1
day_list = [2, 3, 4, 5, 6, 9]

traffic_data_ref = {}

for hour in range(24):
    for minute in range(60):
        for day in day_list:
            key = str(tmc) + '_' + str(month) + '_' + str(day) + '_' + str(hour) + '_' + str(minute)
            traffic_data_ref[key] = float(filtered_tmc_speed_dict[key].split('_')[0])
#         print(hour, minute, float(filtered_tmc_speed_dict[key].split('_')[0]))

zdump(traffic_data_ref, '../temp_files/traffic_data_ref.pkz')